In [1]:
import torch
from torch import nn

In [2]:
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.4299],
        [0.4847]], grad_fn=<AddmmBackward>)

In [5]:
# 查看最后一层的权重
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.1260, -0.0873, -0.0263,  0.3446,  0.1917,  0.0797,  0.3531, -0.0282]])), ('bias', tensor([0.2572]))])


In [6]:
# 查看最后一层的偏移
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.2572], requires_grad=True)
tensor([0.2572])


In [9]:
# 查看最后一层的梯度，还没做反向传播所以还没有记录梯度
net[2].weight.grad == None

True

In [11]:
# 一次性访问一层的所有参数
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
# 一次性访问整个网络的所有参数
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [13]:
# 可以通过名字指定访问参数
net.state_dict()['2.bias'].data

tensor([0.2572])

**从嵌套块收集参数**

In [19]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block1 {i}', block1())    # block2是4个block1嵌套起来，命名
    return net

In [20]:
rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.0232],
        [-0.0231]], grad_fn=<AddmmBackward>)

In [21]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block1 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


**内置参数初始化：可以自定义不同层的初始化方法，并apply到网络的每一层**

In [22]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

In [23]:
net.apply(init_normal)

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)

In [24]:
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0088, -0.0042, -0.0061,  0.0014]), tensor(0.))

In [25]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
        
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

**对某些块应用不同的初始化方法**

In [26]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)
        
net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.6839,  0.0282, -0.5772, -0.5751])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


**自定义初始化**

In [29]:
def my_init(m):
    if type(m) == nn.Linear:
        print(
            "Init", 
            *[(name, param.shape) for name, param in m.named_parameters()][0]
        )
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >=5
    
net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-0.0000, -9.5320, -9.2159,  0.0000],
        [ 0.0000, -0.0000,  6.5028,  0.0000]], grad_fn=<SliceBackward>)

**可以直接访问weight并修改**

In [30]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000, -8.5320, -8.2159,  1.0000])

**参数绑定，做共享的参数**

In [31]:
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), 
                    shared, nn.ReLU(), 
                    shared, nn.ReLU(), 
                    nn.Linear(8, 1))
net(X)

tensor([[-0.0874],
        [-0.0777]], grad_fn=<AddmmBackward>)

In [32]:
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])


In [33]:
net[2].weight.data[0, 0] = 10
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])


一边修改后，另一边也一起改了